<a href="https://colab.research.google.com/github/gitmystuff/DTSC5082/blob/main/Interview_Prep_5/interview_prep_concepts_review_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised, Unsupervised, and Other Learning Paradigms

# Part 1

## 🎯 Objective
The prep assignments are not about getting a high accuracy score. It is about **process** and **interpretation**. In a technical interview, you will rarely be judged solely on your model's RMSE; you will be judged on:
1.  Can you explain *why* you chose that model?
2.  Can you interpret the results for a non-technical stakeholder?
3.  Do you understand the mathematical foundations (e.g., OLS assumptions, Log-Odds)?

70+ points you move on to the next round. < 70 you don't get a call back.

## 📚 Topics Covered
* **Linear Regression:** OLS, Assumptions, and Coefficient Interpretation.
* **GLMs:** Moving beyond the Normal distribution.
* **Logistic Regression:** Probabilities, Thresholds, and Log-Odds.
* **Tree-Based Models:** Decision Trees, Random Forests, and Overfitting.

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
import statsmodels.api as sm
import scipy.stats as stats

# Configuration for prettier plots
sns.set_style("whitegrid")
%matplotlib inline

## The Linear Foundation & Interpretation

### 💡 Concept Review
We start with **Ordinary Least Squares (OLS)**. The goal is to minimize the Sum of Squared Residuals (SSR).
$$\min \sum (y_i - \hat{y}_i)^2$$

### 🎤 Interview Hot Seat
**Interviewer:** "I see you have an $R^2$ of 0.89. What does that actually mean? And why might we look at Adjusted $R^2$ instead?"



In [ ]:
# --- CODE CHALLENGE 1: Interpret the Coefficients ---
# Scenario: We are predicting Sales based on TV, Radio, and Newspaper ad spend.
# Data generation (mimicking the "Advertising" dataset from your notes)

np.random.seed(42)
n_samples = 200
tv = np.random.normal(150, 50, n_samples)
radio = np.random.normal(30, 10, n_samples)
newspaper = np.random.normal(40, 20, n_samples)

# True relationship: Sales = 2.8 + 0.046*TV + 0.19*Radio + noise
sales = 2.8 + 0.046 * tv + 0.19 * radio + np.random.normal(0, 2, n_samples)

df_ads = pd.DataFrame({'TV': tv, 'Radio': radio, 'Newspaper': newspaper, 'Sales': sales})

# 1. Prepare Data for Statsmodels (Add Constant)
X = df_ads[['TV', 'Radio', 'Newspaper']]
y = df_ads['Sales']

# TODO: Add a constant to X (Statsmodels requires this explicitly!)
X_const = sm.add_constant(X)

# 2. Fit OLS Model
model = sm.OLS(y, X_const).fit()

# 3. Print Summary
print(model.summary())

# --- DISCUSSION ---
# Look at the coefficient for 'Radio' (approx 0.19).
# QUESTION: If we spend $1,000 more on Radio ads, how do Sales change?
# HINT: Check the units. If Sales is in units and spend is in dollars...

## Generalized Linear Models (GLMs)

### 💡 Concept Review
Standard Linear Regression assumes the residuals are normally distributed (Gaussian) and the relationship is linear ($\eta = \mu$). But real data is rarely this clean.
* **GLMs** allow us to specify a **Random Component** (distribution like Poisson, Binomial) and a **Link Function**.
* **Link Function:** Transforms the expected mean to fit the linear predictor.
    * Linear Regression: Identity Link ($\eta = \mu$).
    * Logistic Regression: Logit Link ($\eta = \log(\frac{\mu}{1-\mu})$).
    * Poisson Regression: Log Link ($\eta = \log(\mu)$).

### 🎤 Interview Hot Seat
**Interviewer:** "Why can't I use standard OLS regression to predict the number of website visitors per hour?"


## How it Works

| Step | Component | Mathematical Representation | What it Does |
| :--- | :--- | :--- | :--- |
| **1** | **Linear Predictor** | $\eta = \beta_0 + \beta_1X_1 + \dots$ | This is your "score"—the weighted sum of your inputs. |
| **2** | **Link Function** | $g(\mu) = \eta$ | The bridge that connects the score to the expected mean. |
| **3** | **Random Component** | $Y \sim \text{Normal}(\mu, \sigma^2)$ | Adds the "noise" (residuals) to account for real-world variation. |

### Why distinguish between $\eta$ and $\mu$?

In standard linear regression, we often treat them as the same thing because the **Identity Link** doesn't change anything ($\eta = \mu$).

However, the distinction becomes vital when your data isn't a straight line. For example:

* **In Logistic Regression:** Your equation ($\eta$) could result in a value of **10.5**. But a probability ($\mu$) cannot be 10.5; it must be between 0 and 1. The **Logit Link** acts as the bridge that "squashes" that 10.5 into a valid probability.
* **In Poisson Regression:** Your equation ($\eta$) might result in **-2.0**. But you can't have a count ($\mu$) of negative apples. The **Log Link** ensures the final prediction is always positive.

### Transforming the Linear Equation to a Probability (Logistic Regression)

To convert the raw "score" from a linear equation into a probability, we use the **Inverse Link Function** (specifically the **Logistic** or **Sigmoid** function). This ensures the output is always between 0 and 1.

#### The Mathematical Formula
Given the linear predictor $\eta = \beta_0 + \beta_1X_1 + \dots$, the probability $\mu$ is calculated as:

$$\mu = \frac{1}{1 + e^{-\eta}}$$

---

#### Step-by-Step Example: Predicting Exam Success
* **Scenario:** Predicting if a student passes (1) or fails (0) based on study hours.
* **Model Coefficients:** Intercept $\beta_0 = -3$, Slope $\beta_1 = 1$.
* **Input:** Student studied for **5 hours**.

| Step | Operation | Formula / Calculation | Result |
| :--- | :--- | :--- | :--- |
| **1** | **Calculate Score ($\eta$)** | $\eta = -3 + 1(5)$ | **2** |
| **2** | **Apply Exponential** | $e^{-\eta} = e^{-2}$ | **0.135** |
| **3** | **Calculate Probability ($\mu$)** | $\mu = \frac{1}{1 + 0.135}$ | **0.88** |

**Conclusion:** The model predicts an **88% probability** ($\mu = 0.88$) that the student will pass the exam.

In [ ]:
# Why Linear Fits Fail on Curved Data ---

# Generate non-linear data (Polynomial)
X_poly = 6 * np.random.rand(100, 1) - 3
y_poly = 0.5 * X_poly**2 + X_poly + 2 + np.random.normal(0, 1, (100, 1))

plt.figure(figsize=(10, 6))
plt.scatter(X_poly, y_poly, color='blue', label='Data')

# Fit Linear Regression (Underfitting)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y_poly)
y_pred_lin = lin_reg.predict(X_poly)

plt.plot(X_poly, y_pred_lin, color='red', label='Linear Fit (Underfitting)')

# Fit Polynomial Regression (Better Fit)
# In an interview, you might be asked how to fix the red line.
# Answer: "Feature Engineering - adding polynomial terms."
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly_trans = poly_features.fit_transform(X_poly)
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_poly_trans, y_poly)

# Sort for plotting
X_new = np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg_poly.predict(X_new_poly)

plt.plot(X_new, y_new, "g-", linewidth=2, label="Polynomial Degree 2")
plt.legend()
plt.title("Linear vs. Polynomial Regression")
plt.show()


## Logistic Regression (Classification)

### 💡 Concept Review
We move from predicting a value to predicting a class (0 or 1).
To do this, we wrap our linear equation in a **Sigmoid** function to squash the output between 0 and 1.

$$P(y=1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}}$$

### 🎤 Interview Hot Seat
**Interviewer:** "What is the relationship between Probability, Odds, and Log-Odds?"

### 🎤 Interview Hot Seat
**Interviewer:** "Is the decision threshold always 0.5?"



In [ ]:
# The Sigmoid Visualization ---

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-10, 10, 100)
phi_z = sigmoid(z)

plt.figure(figsize=(10, 6))
plt.plot(z, phi_z)
plt.axvline(0.0, color='k')
plt.axhspan(0.0, 1.0, facecolor='1.0', alpha=1.0, ls='dotted')
plt.axhline(y=0.5, ls='dotted', color='k', label='Default Threshold (0.5)')
plt.yticks([0.0, 0.5, 1.0])
plt.ylim(-0.1, 1.1)
plt.title('The Sigmoid Function: Squashing outputs to Probabilities')
plt.xlabel('z (Linear Combination)')
plt.ylabel('$\phi (z)$ (Probability)')
plt.legend()
plt.show()

# Quick Logistic Regression on our Ad Data (converting Sales to High/Low)
# Class 1 = High Sales (> 20), Class 0 = Low Sales
df_ads['High_Sales'] = (df_ads['Sales'] > 20).astype(int)

log_reg = LogisticRegression()
log_reg.fit(df_ads[['TV']], df_ads['High_Sales'])

# Check coefficient
print(f"Coefficient for TV: {log_reg.coef_[0][0]}")
# Interpretation: Positive coef means higher TV spend increases log-odds of having 'High Sales'.


## Decision Trees & Random Forests

### 💡 Concept Review
**Decision Trees** split data based on rules (e.g., if TV Spend > 100). They are prone to **overfitting** (memorizing the noise).
**Random Forests** solve this via **Bagging** (Bootstrap Aggregating):
1.  Train many trees on random subsets of data.
2.  Each tree sees only a subset of features.
3.  Average the results (Regression) or Vote (Classification).

### 🎤 Interview Hot Seat
**Interviewer:** "What are hyperparameters, and how do we tune them?"


In [ ]:
# Grid Search in Action ---

from sklearn.datasets import load_iris

# Load Data
data = load_iris()
X_iris = data.data
y_iris = data.target

# Split
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)

# Define Model
rf = RandomForestClassifier(random_state=42)

# [cite_start]Define Grid (The "Hyperparameter Space") [cite: 5776]
# We are testing different depths and number of estimators
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2]
}

# TODO: Initialize Grid Search with 5-fold cross-validation
# HINT: Look at the variable names above
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)

# Fit
print("Training Grid Search... (this might take a second)")
grid_search.fit(X_train_i, y_train_i)

# Results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Val Score: {grid_search.best_score_:.4f}")

# Final Test
best_rf = grid_search.best_estimator_
y_pred_i = best_rf.predict(X_test_i)
print(f"Test Set Accuracy: {accuracy_score(y_test_i, y_pred_i):.4f}")

## Section Wrap-Up

In this section, we covered the core "Whiteboard" topics for supervised learning:
1.  **Linear Regression:** Interpreting coefficients as unit changes.
2.  **GLMs:** Matching the distribution to the data type.
3.  **Logistic Regression:** Understanding the Sigmoid and thresholds.
4.  **Random Forests:** Using Ensembles to fight overfitting.

**Next Steps:** Review the supporting PDFs for deeper derivations of the math (specifically the "Linear Regression Metrics" and "GLMs" PDFs).


# Part 2

For Part 2, we shift gears from "Predicting the Answer" to "Finding the Question." In interviews, Unsupervised Learning often trips candidates up because there is no clear "accuracy" metric to hide behind. They have to defend *why* a cluster is meaningful.

## Unsupervised Learning & Pattern Discovery

## 🎯 Objective
In Supervised Learning, we had a target variable ($y$). In Unsupervised Learning, we are flying blind. We are looking for **structure** in the data.
**The Interview Trap:** Candidates often run K-Means, get 3 clusters, and stop. A good Data Scientist asks:
1.  "Are these clusters real or just forced?"
2.  "How do we visualize 50 dimensions?"
3.  "What does 'Cluster 1' actually represent in the real world?"

## 📚 Topics Covered
* **The Curse of Dimensionality:** Why distance breaks down in high dimensions.
* **PCA (Principal Component Analysis):** Squashing dimensions while keeping the "story" (variance).
* **K-Means Clustering:** Partitioning data and finding the "Elbow."
* **Evaluation:** Silhouette Scores (because we don't have Accuracy).
* **DBSCAN:** When clusters aren't perfect circles.
`


In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_wine, make_blobs, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples

# Configuration
sns.set_style("whitegrid")
%matplotlib inline


## Topic 1: The Curse of Dimensionality & PCA

### 💡 Concept Review
As you add more features (dimensions), the "volume" of the space increases so fast that the available data becomes sparse. In high dimensions, **Euclidean distance** (the basis of most clustering) loses meaning because every point is roughly equidistant from every other point.

**PCA** rotates the data to align with the directions of maximum variance (information) and discards the directions of noise.

### 🎤 Interview Hot Seat
**Interviewer:** "I have a dataset with 100 features. Should I just feed all of them into K-Means?"


In [ ]:
# Visualizing the "Squash" ---
# Load the Wine Dataset (13 features)
data = load_wine()
X = data.data
y = data.target # We won't use y for training, only for coloring the plot to check our work.
feature_names = data.feature_names

# 1. Standardize the Data (CRITICAL for PCA and K-Means)
# Why? Because 'Proline' is ~1000 and 'Hue' is ~1. PCA follows the largest scale if not normalized.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Apply PCA
pca = PCA(n_components=2) # Squash 13 dimensions down to 2
X_pca = pca.fit_transform(X_scaled)

# 3. Visualization
plt.figure(figsize=(10, 6))
# We color by 'y' just to see if PCA kept the classes separated naturally
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', edgecolor='k', s=60)
plt.title(f'PCA: Reduced 13 dims to 2. Explained Variance: {sum(pca.explained_variance_ratio_):.2f}')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(scatter, label='True Wine Class')
plt.show()

# Print the "Variance Ratio" - How much info did we keep?
print(f"Variance explained by each component: {pca.explained_variance_ratio_}")


## Topic 2: K-Means Clustering

### 💡 Concept Review
K-Means tries to separate data into $K$ groups of equal variance, minimizing the **Inertia** (sum of squared distances to the nearest cluster center).

**The Algorithm:**
1.  Pick $K$ random centroids.
2.  Assign every point to the nearest centroid.
3.  Move the centroid to the average of its assigned points.
4.  Repeat until they stop moving.

### 🎤 Interview Hot Seat
**Interviewer:** "How do you handle categorical data in K-Means?"
* **Better Approach:** Use **K-Modes** (for categorical) or **K-Prototypes** (mixed), or embed the categories into a vector space first.


In [ ]:
# The Elbow Method ---

# Let's pretend we don't know there are 3 wine classes. Let's find K.
inertia = []
range_k = range(1, 10)

for k in range_k:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# Plot
plt.figure(figsize=(8, 4))
plt.plot(range_k, inertia, 'bx-')
plt.xlabel('k (Number of clusters)')
plt.ylabel('Inertia (Sum of Squared Distances)')
plt.title('The Elbow Method: Showing the optimal k')
plt.show()

# DISCUSSION:
# Where is the "Elbow"? (Likely around 3).
# Inertia ALWAYS goes down as K increases. We want the point of diminishing returns.


## Topic 3: Evaluation (Silhouette Score)

### 💡 Concept Review
Since we don't have labels, we can't calculate Accuracy. We use **Silhouette Score**, which measures:
* **Cohesion:** How close is the point to its own cluster?
* **Separation:** How far is the point from the *next nearest* cluster?

Range: -1 (Wrong cluster) to +1 (Perfect cluster). 0 means overlapping.

### 🎤 Interview Hot Seat
**Interviewer:** "Your Elbow plot is ambiguous. It looks like a smooth curve. How do you decide on K?"



In [ ]:
# Silhouette Analysis ---

from sklearn.metrics import silhouette_score

for k in [2, 3, 4, 5]:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    print(f"For n_clusters = {k}, the Silhouette Score is: {score:.3f}")

# Look for the highest score.
# Note: Sometimes K=3 is best, sometimes K=2. In Unsupervised learning, "Truth" is relative.

## Topic 4: When K-Means Fails (DBSCAN)

### 💡 Concept Review
K-Means assumes clusters are **globular** (spherical blobs). It fails on:
* Crescents or moons.
* Concentric circles (donuts).
* Data with lots of noise/outliers.

**DBSCAN** (Density-Based Spatial Clustering) groups points that are close together and marks points in low-density regions as **Noise (-1)**.
* **Epsilon:** The radius to search.
* **Min_Samples:** Minimum points to form a dense region.


In [ ]:
# --- VISUALIZATION: K-Means vs DBSCAN ---
# Generate "Moon" data (Non-spherical)
X_moons, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

# Fit K-Means
kmeans_moon = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_km = kmeans_moon.fit_predict(X_moons)

# Fit DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=5)
labels_db = dbscan.fit_predict(X_moons)

# Plot Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# K-Means Plot
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_km, cmap='plasma')
axes[0].set_title('K-Means (Fails on shapes)')

# DBSCAN Plot
# Note: Points labeled -1 are noise
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_db, cmap='plasma')
axes[1].set_title('DBSCAN (Captures shapes)')

plt.show()


## Section Wrap-Up

In this section, we explored the "Dark Arts" of Unsupervised Learning:

1.  **PCA:** Pre-processing to remove noise and curse of dimensionality.

2.  **K-Means:** The workhorse for spherical clusters.

3.  **Silhouette Score:** How we grade ourselves without an answer key.

4.  **DBSCAN:** The alternative for complex shapes and outlier detection.

# Part 3

## Advanced Learning Paradigms

## 🎯 Objective
We have covered Supervised (Labeled) and Unsupervised (Unlabeled) learning. Now we look at **how** we learn when the data is scarce, decentralized, or requires interaction.
* **Reinforcement Learning:** Learning by trial and error (Interaction).
* **Transfer Learning:** Learning from someone else's experience (Reuse).
* **Federated Learning:** Learning without seeing the data (Privacy).
* **Semi-Supervised:** Learning with a tiny bit of help (Efficiency).

## 📚 Topics Covered
* **Markov Decision Processes (MDPs):** States, Actions, Rewards.
* **Q-Learning:** The Bellman Equation in code.
* **Pre-training & Fine-tuning:** The "Warm Start."
* **FedAvg:** Averaging weights from decentralized devices.

In [ ]:
# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Configuration
sns.set_style("whitegrid")
%matplotlib inline


## Topic 1: Reinforcement Learning (RL)

## 💡 Concept Review
RL is about an **Agent** interacting with an **Environment** to maximize a **Reward**.

**The MDP Tuple: $(S, A, P, R, \gamma)$**

In Reinforcement Learning, the **Markov Decision Process (MDP)** provides the mathematical framework for modeling decision-making where outcomes are partly random and partly under the control of a decision-maker (the agent).

| Symbol | Name | Mathematical Form | Description |
| :--- | :--- | :--- | :--- |
| **$S$** | **States** | $s \in S$ | The set of all possible "situations" or configurations of the environment. |
| **$A$** | **Actions** | $a \in A$ | The set of all possible moves or decisions the agent can make. |
| **$P$** | **Transition Probability** | $P(s' \mid s, a)$ | The "physics" of the world: the probability of landing in state $s'$ after taking action $a$ in state $s$. |
| **$R$** | **Reward Function** | $R(s, a)$ | The immediate scalar feedback (score) received after taking an action. |
| **$\gamma$** | **Discount Factor** | $\gamma \in [0, 1]$ | Determines the present value of future rewards (how "farsighted" the agent is). |

---

### Deep Dive on $P$ (Transition Dynamics)
The **$P$** component is critical because it defines whether an environment is **Deterministic** or **Stochastic**:

* **Deterministic:** If you take action $a$ in state $s$, you *always* end up in $s'$. ($P = 1$)
* **Stochastic (Probabilistic):** Even if you take action $a$, there is a distribution of possible next states. For example, a robot trying to move forward on ice might have a $0.8$ probability of moving forward and $0.1$ probability of slipping left or right.



---

### The Goal of the Agent
The agent's objective is to find a **Policy** ($\pi$), which is a mapping from states to actions ($\pi: S \rightarrow A$), that maximizes the **Expected Return**:

$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots$$

*Note: This return formula uses an **Exponential Decay** via the Discount Factor ($\gamma$) to ensure that rewards in the far future are worth less than immediate rewards.*

### The Q-Learning Update Rule (The "Interviewer's Favorite Equation")
$$Q(s,a) \leftarrow Q(s,a) + \alpha [R + \gamma \max_{a'} Q(s', a') - Q(s,a)]$$
We update our current estimate of "Quality" ($Q$) based on the reward we just got plus the best possible future reward.

### 🎤 Interview Hot Seat
**Interviewer:** "What is the difference between Exploration and Exploitation?"


In [ ]:
# --- CODE SIMULATION: The "Cliff Walking" GridWorld ---
# We will build a simple environment and solve it with Q-Learning.

class GridWorld:
    def __init__(self):
        self.height = 3
        self.width = 4
        self.goal = (0, 3) # Top right
        self.trap = (1, 3) # Right below goal
        self.state = (2, 0) # Start at bottom left

    def step(self, action):
        # Actions: 0=Up, 1=Down, 2=Left, 3=Right
        y, x = self.state
        if action == 0: y = max(0, y - 1)
        elif action == 1: y = min(self.height - 1, y + 1)
        elif action == 2: x = max(0, x - 1)
        elif action == 3: x = min(self.width - 1, x + 1)

        new_state = (y, x)
        self.state = new_state

        # Rewards
        if new_state == self.goal: return new_state, 10, True  # Win!
        if new_state == self.trap: return new_state, -10, True # Dead!
        return new_state, -1, False # Living cost (encourages speed)

    def reset(self):
        self.state = (2, 0)
        return self.state

# Q-Learning
env = GridWorld()
q_table = np.zeros((env.height, env.width, 4)) # 3x4 grid, 4 actions
alpha = 0.1  # Learning Rate
gamma = 0.9  # Discount Factor
epsilon = 0.1 # Exploration Rate

# Training Loop
for episode in range(500):
    state = env.reset()
    done = False

    while not done:
        # Epsilon-Greedy Strategy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(0, 4) # Explore
        else:
            action = np.argmax(q_table[state[0], state[1]]) # Exploit

        next_state, reward, done = env.step(action)

        # Update Q-Value (Bellman Equation)
        old_value = q_table[state[0], state[1], action]
        next_max = np.max(q_table[next_state[0], next_state[1]])
        new_value = old_value + alpha * (reward + gamma * next_max - old_value)

        q_table[state[0], state[1], action] = new_value
        state = next_state

# Visualization of the Learned Policy
print("Final Q-Table (Best Action per Cell):")
actions = ["↑", "↓", "←", "→"]
grid = [['' for _ in range(4)] for _ in range(3)]
for y in range(3):
    for x in range(4):
        best_action = np.argmax(q_table[y, x])
        grid[y][x] = actions[best_action]

grid[0][3] = "GOAL"
grid[1][3] = "TRAP"
for row in grid:
    print(row)


## Topic 2: Transfer Learning

### 💡 Concept Review
Training Deep Learning models from scratch is expensive (weeks of GPU time).
**Transfer Learning** allows us to take a model trained on a large dataset (e.g., ImageNet) and "fine-tune" it on a smaller, specific dataset.

**Strategies:**
1.  **Feature Extraction:** Freeze the backbone (the complex part), only retrain the last layer (the classifier).
2.  **Fine-Tuning:** Unfreeze everything but use a very small learning rate to adjust weights slightly.
3. https://www.researchgate.net/figure/The-architecture-of-our-transfer-learning-model_fig4_342400905

### 🎤 Interview Hot Seat
**Interviewer:** "I have a small dataset of medical images (1000 X-rays). Should I train a ResNet-50 from scratch?"


In [ ]:
# TRANSFER LEARNING DEMO WITH EXPLANATORY PRINTS

from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


# ------------------------------------------------------------
# STEP 1: Generate Dataset
# ------------------------------------------------------------

print("\nSTEP 1: Generating synthetic dataset...\n")

X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_classes=4,
    n_informative=10,
    random_state=42
)

print("Dataset shape:", X.shape)
print("Unique classes:", set(y))


# ------------------------------------------------------------
# STEP 2: Split into Source and Target Tasks
# ------------------------------------------------------------

print("\nSTEP 2: Splitting into Source Task (0,1) and Target Task (2,3)\n")

mask_source = (y < 2)
mask_target = (y >= 2)

X_source, y_source = X[mask_source], y[mask_source]
X_target, y_target = X[mask_target], y[mask_target]

print("Source samples:", len(y_source))
print("Target samples:", len(y_target))


# ------------------------------------------------------------
# STEP 3: Create Model
# ------------------------------------------------------------

print("\nSTEP 3: Creating Neural Network\n")

mlp = MLPClassifier(hidden_layer_sizes=(10,), random_state=42)

print("Model created.")
print("Hidden layer size:", mlp.hidden_layer_sizes)


# ------------------------------------------------------------
# STEP 4: Initialize Model with ALL classes
# ------------------------------------------------------------

print("\nSTEP 4: Initializing model with ALL classes\n")

all_classes = [0,1,2,3]

mlp.partial_fit(X_source, y_source, classes=all_classes)

print("Classes known to model:", mlp.classes_)
print("Output layer size:", len(mlp.classes_))


# ------------------------------------------------------------
# STEP 5: Train on Source Task
# ------------------------------------------------------------

print("\nSTEP 5: Training on Source Task\n")

epochs = 50

for i in range(epochs):

    mlp.partial_fit(X_source, y_source)

    if i in [0, 1, 5, 10, 25, 49]:

        acc = mlp.score(X_source, y_source)

        print(f"Epoch {i+1:2d}  Source Accuracy: {acc:.3f}")


print("\nFinal Source Accuracy:", mlp.score(X_source, y_source))


# ------------------------------------------------------------
# STEP 6: TRANSFER LEARNING BEGINS HERE
# ------------------------------------------------------------

print("\nSTEP 6: Transfer Learning Begins (Fine-Tuning on Target Task)\n")

print("IMPORTANT: We are NOT creating a new model.")
print("We are continuing training with EXISTING learned weights.\n")


epochs_target = 50

for i in range(epochs_target):

    mlp.partial_fit(X_target, y_target)

    if i in [0, 1, 2, 5, 10, 25, 49]:

        acc = mlp.score(X_target, y_target)

        print(f"Epoch {i+1:2d}  Target Accuracy: {acc:.3f}")


print("\nFinal Target Accuracy (Transfer Learning):",
      mlp.score(X_target, y_target))


# ------------------------------------------------------------
# STEP 7: Train From Scratch (Control Group)
# ------------------------------------------------------------

print("\nSTEP 7: Training Separate Model From Scratch\n")

mlp_scratch = MLPClassifier(
    hidden_layer_sizes=(10,),
    max_iter=50,
    random_state=42
)

mlp_scratch.fit(X_target, y_target)

print("\nScratch Model Target Accuracy:",
      mlp_scratch.score(X_target, y_target))


# ------------------------------------------------------------
# STEP 8: FINAL COMPARISON
# ------------------------------------------------------------

print("\nSTEP 8: FINAL COMPARISON\n")

print("Transfer Learning Accuracy:",
      mlp.score(X_target, y_target))

print("Scratch Accuracy:",
      mlp_scratch.score(X_target, y_target))


## Topic 3: Federated Learning

### 💡 Concept Review
Standard ML requires collecting all data into one central server. This raises **Privacy** and **Bandwidth** concerns (e.g., typing prediction on phones).
**Federated Learning (FedAvg)** reverses this:
1.  Send the *Model* to the device (Client).
2.  Train locally on private data.
3.  Send only the *Weights* (updates) back to the server.
4.  Average the weights to create a new Global Model.

### 🎤 Interview Hot Seat
**Interviewer:** "If I use Federated Learning, can I guarantee user privacy?"




In [ ]:
# --- CODE SIMULATION: Federated Averaging (FedAvg) ---

# Simulate Global Data
X_fed, y_fed = make_classification(n_samples=1000, n_features=5, random_state=42)
X_test_fed = X_fed[-200:]
y_test_fed = y_fed[-200:]

# Simulate 3 Clients (Data Silos)
client_1_X, client_1_y = X_fed[0:200], y_fed[0:200]
client_2_X, client_2_y = X_fed[200:400], y_fed[200:400]
client_3_X, client_3_y = X_fed[400:600], y_fed[400:600]

# Initialize 3 Local Models (Identical Architecture)
model_1 = SGDClassifier(loss='log_loss', random_state=42)
model_2 = SGDClassifier(loss='log_loss', random_state=42)
model_3 = SGDClassifier(loss='log_loss', random_state=42)

# Train Locally (Server sees NONE of this data)
model_1.partial_fit(client_1_X, client_1_y, classes=[0,1])
model_2.partial_fit(client_2_X, client_2_y, classes=[0,1])
model_3.partial_fit(client_3_X, client_3_y, classes=[0,1])

# The Server Aggregation Step (FedAvg)
# We average the coefficients (weights) and intercepts
avg_coef = (model_1.coef_ + model_2.coef_ + model_3.coef_) / 3
avg_intercept = (model_1.intercept_ + model_2.intercept_ + model_3.intercept_) / 3

# Create Global Model with Averaged Weights
global_model = SGDClassifier(loss='log_loss', random_state=42)
global_model.partial_fit(X_test_fed[:5], y_test_fed[:5], classes=[0,1]) # Initialize structure
global_model.coef_ = avg_coef
global_model.intercept_ = avg_intercept

# Evaluate
print(f"Client 1 Accuracy: {model_1.score(X_test_fed, y_test_fed):.2f}")
print(f"Global Model (Federated) Accuracy: {global_model.score(X_test_fed, y_test_fed):.2f}")

# DISCUSSION:
# The Global Model performs well without ever seeing the full training set in one place!


## Topic 4: Semi-Supervised Learning (Label Propagation)

### 💡 Concept Review
What if you have 1,000,000 images but only 100 are labeled?
**Label Propagation** assumes that points close to each other in high-dimensional space likely share the same label. It "spreads" the known labels to nearby unknown neighbors.

**Code Tip:** In `sklearn`, we use `-1` to denote unlabeled data.



In [ ]:
# --- CODE DEMO: Label Propagation ---
from sklearn.semi_supervised import LabelSpreading

# Generate Data
X_semi, y_semi = make_classification(n_samples=200, n_features=2, n_redundant=0, random_state=42)

# Create "Unlabeled" Dataset
y_train_mixed = np.copy(y_semi)
# Mask 95% of labels (Set to -1)
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(y_semi)) < 0.95
y_train_mixed[random_unlabeled_points] = -1

print(f"Total Points: {len(y_semi)}")
print(f"Labeled Points: {len(y_semi) - np.sum(random_unlabeled_points)}")

# Train Label Spreading Model
lp_model = LabelSpreading(kernel='knn', alpha=0.8)
lp_model.fit(X_semi, y_train_mixed)

# Evaluate on the ground truth of the masked points
predicted_labels = lp_model.transduction_[random_unlabeled_points]
true_labels = y_semi[random_unlabeled_points]

print(f"Accuracy on Unlabeled Data: {accuracy_score(true_labels, predicted_labels):.2f}")

# Plot
plt.figure(figsize=(10, 6))
# Plot unlabeled (now predicted)
plt.scatter(X_semi[random_unlabeled_points, 0], X_semi[random_unlabeled_points, 1],
            c=predicted_labels, cmap='viridis', marker='.', label='Propagated Labels')
# Plot original labeled
plt.scatter(X_semi[~random_unlabeled_points, 0], X_semi[~random_unlabeled_points, 1],
            c=y_semi[~random_unlabeled_points], cmap='viridis', marker='s', s=100, edgecolor='k', label='Original Labeled')
plt.legend()
plt.title("Semi-Supervised Learning: Spreading 5% labels to 95% data")
plt.show()


## Section Wrap-Up

In this section, we stepped outside the traditional "Fit-Predict" loop.

1.  **RL:** We built an agent that learns from *consequences*.

2.  **Transfer:** We learned how to stand on the shoulders of giants (pre-trained models).

3.  **Federated:** We learned how to train without centralizing data (privacy).

4.  **Semi-Supervised:** We learned how to leverage unlabeled data.

